### Simulation

* make 4 sets of sequences: 250 seq , 1000 seqs, 5000 seqs, each 250 bp (meme takes 60000 only)
    * Use random generator with real frequencies
* Insert real motifs but have been modified to have mC.
    * CEBPB , NRF1, SP1, CTCF, made up motif
* Different abundances: 1%, 2%, 5%, 10%, 25% of peaks
    

In [1]:
abundances = [0.01,0.02,0.05,0.1,0.25]
sizes = [250, 1000, 5000]
seq_length_avg = 250
seq_length_std = 50




In [67]:
import os 
from sys import argv
import random as rd
from copy import deepcopy

def load_motifsEF(filename):
    file=open(filename)
    seq=file.read().split("MOTIF")
    seq=seq[1:]
    motifs={}
    infos={}
    for s in range(len(seq)):
        t=seq[s].strip().split("\n")
        motifs[t[0]]=t[2:]  #remove HOCOMOCO URL
        infos[t[0].split('_')[0]]=t[:2]
    for m in motifs:
        tdict={'A':[],'C':[],'G':[],'T':[],'E':[],'F':[]}
        for pos in range(len(motifs[m])):
            tmp=motifs[m][pos].strip().split("\t")
            tdict['A']+=[float(tmp[0])]
            tdict['C']+=[float(tmp[1])]
            tdict['G']+=[float(tmp[2])]
            tdict['T']+=[float(tmp[3])]
            tdict['E']+=[float(tmp[4])]
            tdict['F']+=[float(tmp[5])]
        motifs[m]=tdict
    
    return motifs,infos


def load_motifs(filename):
    file=open(filename)
    seq=file.read().split("MOTIF")
    seq=seq[1:]
    motifs={}
    infos={}
    for s in range(len(seq)):
        t=seq[s].strip().split("\n")
        motifs[t[0]]=t[2:-1]  #remove HOCOMOCO URL
        infos[t[0].split('_')[0]]=t[:2]
    for m in motifs:
        tdict={'A':[],'C':[],'G':[],'T':[]}
        for pos in range(len(motifs[m])):
            tmp=motifs[m][pos].strip().split("\t")
            tdict['A']+=[float(tmp[0])]
            tdict['C']+=[float(tmp[1])]
            tdict['G']+=[float(tmp[2])]
            tdict['T']+=[float(tmp[3])]
            #tdict['E']+=[float(tmp[4])]
            #tdict['F']+=[float(tmp[5])]
        motifs[m]=tdict
    
    return motifs,infos
def insertBaseToPWM(pwm,loc,prop):
    #randomly insert modified bases to an existing pwm
    #insert at loc location, probability is prop
    newpwm = deepcopy(pwm)
    k = len(pwm['A'])
    base = rd.choice(['E','E']) # only adds E, for this is typeE
    freq = prop #rd.choice(range(5,10))*0.1 #prop 0.5 - 1.0
    print base,loc,freq
    newpwm['E'] = [0]*k #add other empty columns to the matrix
    newpwm['F'] = [0]*k

    for c in newpwm.keys():
        newpwm[c][loc] = (1-freq)/(len(newpwm.keys())-1)
    newpwm[base][loc] = freq
    return newpwm

def randomkmer_typeEF(PWM):
    #take a PWM and randomly create a Kmer based on it
    # chance goes A->C->G->T->E
    kmer=""
    for pos in range(len(PWM['A'])):
        die=rd.randint(0,100000)
        if die<=PWM['A'][pos]*100000:
            kmer+='A'
        elif die<=(PWM['A'][pos]+PWM['C'][pos])*100000:
            kmer+='C'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos])*100000:
            kmer+='G'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos]+PWM['T'][pos])*100000:
            kmer+='T'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos]+PWM['T'][pos]+PWM['E'][pos])*100000:
            kmer+='E'
        else:
            kmer+='F'
    return kmer

def loadFasta(filename):
    seqs = {}
    lines = open(filename).read().split('>')[1:]
    for line in lines:
        tmp = line.split('\n')
        seqs[tmp[0]] = ''.join(tmp[1:])
    return seqs

def insertoseqs(pwm,seqs,numseqtopick):
    #take pwm, the seqs, and insert the pwm to numseqtopick of the seqs 
    
    seqspicked = rd.sample(seqs.keys(),numseqtopick)
    #print seqspicked
    for s in seqspicked:
        kmer = randomkmer_typeEF(pwm)
        #print kmer
        string = list(seqs[s])
        insertloc = rd.randrange(0,len(string)-len(kmer))
        #print insertloc
        for i in xrange(len(kmer)):
            string[insertloc + i]=kmer[i]
    
        seqs[s] = ''.join(string)
    return seqs
def printPWM(pwm,name,outfile):
    target = open(outfile,'w')
    target.write("MOTIF\t"+name+'\n')
    for loc in range(len(pwm['A'])):
        line = []
        for base in ['A','C','G','T','E','F']:
            line += [str(pwm[base][loc])]
        line = '\t'.join(line)
        target.write(line+'\n')
    target.close()
    return
def getConsensus(pwm):
    #take the dict pwm and print the consensus of the pwm
    cnss = '' 
    for loc in range(len(pwm['A'])):
        maxbase = 'A'
        maxvalue = 0.0
        for base in pwm:
            if pwm[base][loc] > maxvalue:
                maxbase = base
                maxvalue = pwm[base][loc]
        cnss += maxbase
    return cnss

In [59]:
knownMotifs,knownInfos = load_motifs("HOCOMOCOv11_core_HUMAN_mono_meme_format.meme")

In [68]:
mymotifs = 
for key in knownMotifs:
    if "CTCF_" in key or "CEBPB" in key or "NRF1" in key:
        print key
        #print knownMotifs[key]
        print getConsensus(knownMotifs[key])
        


CTCF_HUMAN.H11MO.0.A
TGGCCACCAGGGGGCGCCA
CEBPB_HUMAN.H11MO.0.A
GATTGTGCAATA
NRF1_HUMAN.H11MO.0.A
CACTGCGCATGCGCAGG
